In [ ]:
# Extract data from API and return json
# Output data to csv
# Read json , do some transform and load to database ( mysql )
# build data model -> view ( DBT )

In [1]:
import pandas as pd
import requests
import time
import json


In [2]:
url = "https://api-football-v1.p.rapidapi.com/v3/"
end_point = {
     'players': {'url': 'players', 'page': 42, 'params': {'league':'39' ,'season':'2024','page': 42} }
    ,'leagues': {'url': 'leagues'} # done
    ,'teams': {'url': 'teams', 'params': {'league':39,'season':'2024'} } #done
    ,'coachs': {'url': 'coachs', 'params' : {'team': 33}} # done

}    


with open('key.json', 'r') as file:
    data = json.load(file)


duckdb_connection = data['duckdb_connection_path'] # CHANGE YOUR DATABASE CONNECTION HERE
'''
Why duckdb Database:
    1. Duckdb is OLAP database , columnar store -> fast retrieve & aggregate data
    2. Easy setup
    3. Strong in support parsing STRUCT | OBJECT dtype format to flatten table 
    4. Common use in transform layer before write to target destination
    5. SQL friendlier
'''


headers = data['headers'] # CHANGE YOUR API KEY HERE



In [23]:
def extract(data,ip_url,ip_end_point, ip_header,ip_page = 1):
	
	url = ip_url + ip_end_point[data]['url'] 
	querystring = ip_end_point[data].get('params')
	page = ip_end_point[data].get('page')

	if page is None:
		page = ip_page
		
	if querystring is None:
		querystring = {}

	raw_data = []
	for i in range(1,page + 1):
		if data == 'players':
			querystring = {'league':'39' ,'season':'2024','page': i}
			response = requests.get(url, headers=ip_header , params=querystring)  #{'league':'39' ,'season':'2024','page': 42}
		else:
			response = requests.get(url, headers=ip_header , params=querystring)  
		raw = response.json()
		raw_data.append(raw)
		print(i)
		time.sleep(5) # add sleep between each call 
	return raw_data

In [4]:
league_data = extract('leagues'
                      ,ip_url=url
                      ,ip_end_point=end_point
                      ,ip_header=headers )


1


In [5]:
teams_data = extract('teams'
                      ,ip_url=url
                      ,ip_end_point=end_point
                      ,ip_header=headers )


1


In [8]:
coachs_data = extract('coachs'
                      ,ip_url=url
                      ,ip_end_point=end_point
                      ,ip_header=headers )


1


In [24]:
players_data = extract('players'
                      ,ip_url=url
                      ,ip_end_point=end_point
                      ,ip_header=headers )


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [33]:
import duckdb

def transform(data, table_name):
    if table_name == 'players_tbl':
        df = pd.DataFrame(data)
    else:
       df = pd.DataFrame(data[0]['response'])
    # Load to DuckDB
    with duckdb.connect(duckdb_connection) as conn:
        conn.sql(f'drop table if exists  {table_name};')
        conn.sql(f'create table {table_name} as select * from df ;')
        # conn.sql('show tables;')


    print('Sucess load to duckdb')


In [12]:
transform(league_data,table_name='league_tbl')

q = '''
select  
-- id::varchar as league_id
--,name as league_name
--,type as league_type
--,logo as league_logo
--,name_1 as country_name
--,code as country_code
--,flag as country_flag
--,year as season_year
--,"start" as season_start
--,"end" as season_end 
--, 
* EXCLUDE (events, lineups, statistics_fixtures, statistics_players, standings, players, top_scorers, top_assists, top_cards, injuries, predictions,odds)


from 
(
select 
 UNNEST (league, RECURSIVE:= true )
,UNNEST (country, RECURSIVE:= true)
,UNNEST (seasons, RECURSIVE:= true), generate_subscripts(seasons , 1) AS index
 
from sampledb.main.league_tbl 

) a
'''
with duckdb.connect(duckdb_connection) as conn:
    conn.sql(q).to_csv('data/league_tbl.csv')
    print('write to league_tbl.csv DONE')
    


Sucess load to duckdb
None


In [13]:
transform(teams_data,'team_tbl')
q = '''
select * 
from 
(

select 
	   UNNEST (team , RECURSIVE:= true )
	 , UNNEST (venue , RECURSIVE:= true )
	 
from sampledb.main.team_tbl
) a
'''
with duckdb.connect(duckdb_connection) as conn:
    conn.sql(q).to_csv('data/team_tbl.csv')
    print('write to team_tbl.csv DONE')
    


Sucess load to duckdb
None


In [14]:
transform(coachs_data,table_name='coach_tbl')

q = '''
select 
	id
	,name
	,firstname
	,lastname 
	,age
	,UNNEST (birth , RECURSIVE:= true )
	,nationality 
	,height 
	,weight 
	,photo 
	,UNNEST (team , RECURSIVE:= true )
	,UNNEST (career , RECURSIVE:= true )
	

from sampledb.main.coach_tbl
'''
with duckdb.connect(duckdb_connection) as conn:
    conn.sql(q).to_csv('data/coach_tbl.csv')
    print('write to coach_tbl.csv DONE')


Sucess load to duckdb
None


In [36]:
transform(players_data,table_name='players_tbl')

q = '''

SELECT *   EXCLUDE (statistics)
from
(
select 
  UNNEST (paging , RECURSIVE:= true  )  
, UNNEST (response , RECURSIVE:= true ) 

from sampledb.main.players_tbl
) a

'''
with duckdb.connect(duckdb_connection) as conn:
    conn.sql(q).to_csv('data/players_tbl.csv')
    print('write to players_tbl.csv DONE')


Sucess load to duckdb
None
